In [8]:
## imports
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torchvision.datasets import Flowers102
from tqdm import tqdm

In [9]:
## -------------------------
## Inception Module Variants
## -------------------------

## 1️⃣ Inception-v1 (Basic GoogleNet Module)
class InceptionV1(nn.Module):
    def __init__(self, in_channels):
        super(InceptionV1, self).__init__()

        ## 1x1 Convolution
        self.conv1x1 = nn.Conv2d(in_channels, 64, kernel_size=1)

        ## 3x3 Convolution
        self.conv3x3 = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )

        ## 5x5 Convolution (Expensive in computation)
        self.conv5x5 = nn.Sequential(
            nn.Conv2d(in_channels, 32, kernel_size=5, padding=2),
            nn.ReLU()
        )

        ## Max Pooling + 1x1 Convolution
        self.maxpool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, 32, kernel_size=1),
            nn.ReLU()
        )

    def forward(self, x):
        return torch.cat([
            self.conv1x1(x),
            self.conv3x3(x),
            self.conv5x5(x),
            self.maxpool(x)
        ], dim=1)

## 2️⃣ Inception-v2 (Factorized 5x5 -> Two 3x3 for Efficiency)
class InceptionV2(nn.Module):
    def __init__(self, in_channels):
        super(InceptionV2, self).__init__()

        ## 1x1 Convolution
        self.conv1x1 = nn.Conv2d(in_channels, 64, kernel_size=1)

        ## Factorized 5x5 -> Two 3x3
        self.conv_factorized = nn.Sequential(
            nn.Conv2d(in_channels, 64, kernel_size=3, padding=1),
            nn.ReLU(),
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )

        ## Normal 3x3 Convolution
        self.conv3x3 = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )

        ## Max Pooling + 1x1 Convolution
        self.maxpool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, 32, kernel_size=1),
            nn.ReLU()
        )

    def forward(self, x):
        return torch.cat([
            self.conv1x1(x),
            self.conv_factorized(x),
            self.conv3x3(x),
            self.maxpool(x)
        ], dim=1)

## 3️⃣ Inception-v3 (Asymmetric Convolutions for Efficiency)
class InceptionV3(nn.Module):
    def __init__(self, in_channels):
        super(InceptionV3, self).__init__()

        ## 1x1 Convolution
        self.conv1x1 = nn.Conv2d(in_channels, 64, kernel_size=1)

        ## Asymmetric 3x3 -> (1x3 + 3x1)
        self.conv_asymmetric = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=(1, 3), padding=(0, 1)),
            nn.ReLU(),
            nn.Conv2d(128, 128, kernel_size=(3, 1), padding=(1, 0)),
            nn.ReLU()
        )

        ## Normal 3x3 Convolution
        self.conv3x3 = nn.Sequential(
            nn.Conv2d(in_channels, 128, kernel_size=3, padding=1),
            nn.ReLU()
        )

        ## Max Pooling + 1x1 Convolution
        self.maxpool = nn.Sequential(
            nn.MaxPool2d(kernel_size=3, stride=1, padding=1),
            nn.Conv2d(in_channels, 32, kernel_size=1),
            nn.ReLU()
        )

    def forward(self, x):
        return torch.cat([
            self.conv1x1(x),
            self.conv_asymmetric(x),
            self.conv3x3(x),
            self.maxpool(x)
        ], dim=1)


In [10]:

## Device Configuration
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

## Data Transforms (Resize + Normalize)
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.5, 0.5, 0.5], std=[0.5, 0.5, 0.5])
])

## Load Flowers-102 Dataset
train_dataset = Flowers102(root="./data", split="train", transform=transform, download=True)
test_dataset = Flowers102(root="./data", split="test", transform=transform, download=True)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=32, shuffle=False)

In [ ]:

## -------------------------
## Full GoogleNet Model (Combining Modules)
## -------------------------
class GoogleNetCustom(nn.Module):
    def __init__(self, num_classes=102):
        super(GoogleNetCustom, self).__init__()
        self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3)
        self.pool1 = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)

        ## Adding Different Inception Modules
        self.inception1 = InceptionV1(64)
        self.inception2 = InceptionV2(256)
        self.inception3 = InceptionV3(352)

        self.pool2 = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(352, num_classes)  ## Adjusted for concatenated outputs

    def forward(self, x):
        x = self.pool1(torch.relu(self.conv1(x)))
        x = self.inception1(x)
        x = self.inception2(x)
        x = self.inception3(x)
        x = self.pool2(x)
        x = torch.flatten(x, 1)
        x = self.fc(x)
        return x

## Instantiate and Move Model to Device
model = GoogleNetCustom().to(device)

## Loss and Optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

## Train Model (Basic Loop)
def train_model():
    model.train()
    for epoch in tqdm(range(5)):  ## Reduce epochs for quick training
        running_loss = 0.0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(images)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()
            running_loss += loss.item()
        print(f"Epoch {epoch+1}, Loss: {running_loss / len(train_loader):.4f}")

train_model()
